In [1]:
#Option Pricing Model Using A Galton Board, for CALLS
import math
import numpy as np
from scipy.stats import norm

# Option pricing function using the BSM model
def bsm_option_price(spot, strike, rate, vol, time_to_expiry, option_type):
    d1 = (math.log(spot / strike) + (rate + 0.5 * vol**2) * time_to_expiry) / (vol * math.sqrt(time_to_expiry))
    d2 = d1 - vol * math.sqrt(time_to_expiry)
    if option_type == 'call':
        price = spot * norm.cdf(d1) - strike * math.exp(-rate * time_to_expiry) * norm.cdf(d2)
    else:
        price = strike * math.exp(-rate * time_to_expiry) * norm.cdf(-d2) - spot * norm.cdf(-d1)
    return price

# Galton board function
def simulate_galton_board(num_balls, num_slots, volatility, time_to_expiry, current_price, rate):
    slots = [0] * (num_slots + 1)
    for i in range(num_balls):
        pos = 0
        for j in range(num_slots - 1):
            if np.random.rand() < 0.5:
                pos += 1
        slots[pos] += 1 if pos <= num_slots else 0
    end_prices = [current_price * math.exp((rate - 0.5 * volatility**2) * time_to_expiry + volatility * math.sqrt(time_to_expiry) * ((i - num_slots/2) / num_slots)) for i in range(num_slots+1)]
    return slots, end_prices

# Example usage
spot = 39.8
strike = 39.5
rate = 0.08
volatility = 0.2
time_to_expiry = 5
num_balls = 10000
num_slots = 100
option_type = 'call'

# Calculate the option price using the BSM model
bsm_price = bsm_option_price(spot, strike, rate, volatility, time_to_expiry, option_type)

# Simulate possible future prices of the underlying asset using the Galton board function
slots, prices = simulate_galton_board(num_balls, num_slots, volatility, time_to_expiry, spot, rate)

# Estimate the probability distribution of the future asset price
price_distribution = np.zeros(num_slots+1)
for i in range(num_slots+1):
    price_distribution[i] = slots[i] / num_balls

# Calculate the expected future price
expected_price = np.sum(price_distribution * prices)

# Calculate the option payoff
if option_type == 'call':
    option_payoff = max(expected_price - strike, 0)
else:
    option_payoff = max(strike - expected_price, 0)

# Calculate the option price based on the estimated probability distribution
estimated_price = option_payoff * math.exp(-rate * time_to_expiry)

# Print the results
print("BSM Price: ", bsm_price)
print("Estimated Price: ", estimated_price)


BSM Price:  14.728525560763249
Estimated Price:  9.44843129945016


In [2]:
#Option Pricing Model Using Galton Board, for PUTS
import random
from scipy.stats import norm

# Galton board function
def simulate_galton_board(asset_price, drift, volatility, time_horizon, num_simulations):
    if not isinstance(asset_price, (float, int)):
        raise TypeError('asset_price must be a numeric value')

    dt = time_horizon / 365
    asset_prices = [asset_price] * num_simulations

    for i in range(num_simulations):
        for j in range(int(time_horizon / dt)):
            asset_prices[i] *= 1 + drift * dt + volatility * norm.ppf(random.random()) * (dt ** 0.5)

    return asset_prices


def payoff(asset_price, strike_price):
    return max(strike_price - asset_price, 0)


def option_price(asset_price, strike_price, risk_free_rate, time_to_expiry, num_simulations=10000, num_slots=100):
    dt = time_to_expiry / num_slots
    asset_prices = [asset_price] * num_simulations
    for i in range(num_slots):
        slots = simulate_galton_board(num_simulations, i)
        for j in range(num_simulations):
            asset_prices[j] *= (1 + (0.05 - 0.5 * 0.2 ** 2) * dt + 0.2 * norm.ppf(slots[j] / num_simulations) * (dt ** 0.5))
    payoffs = [payoff(ap, strike_price) for ap in asset_prices]
    discount_factor = 1 / (1 + risk_free_rate * time_to_expiry)
    return discount_factor * sum(payoffs) / num_simulations





# Example usage:
asset_price = 32
strike_price = 34
risk_free_rate = 0.02
time_to_expiry = 0.5
num_simulations = 20
drift = .1
time_horizon = 2
option_type = "put"


# Calculate the option price using the BSM model
bsm_price = bsm_option_price(spot, strike, rate, volatility, time_to_expiry, option_type)

# Simulate possible future prices of the underlying asset using the Galton board function
slots, prices = simulate_galton_board(asset_price, drift, volatility, time_horizon, num_simulations)

# Estimate the probability distribution of the future asset price
price_distribution = np.zeros(num_slots+1)
for i in range(num_slots+1):
    price_distribution[i] = slots[i] / num_balls

# Calculate the expected future price
expected_price = np.sum(price_distribution * prices)

# Calculate the option payoff
if option_type == 'call':
    option_payoff = max(expected_price - strike, 0)
else:
    option_payoff = max(strike - expected_price, 0)

# Calculate the option price based on the estimated probability distribution
estimated_price = option_payoff * math.exp(-rate * time_to_expiry)

# Print the results
print("BSM Price: ", bsm_price)
print("Estimated Price: ", estimated_price)

ValueError: too many values to unpack (expected 2)

In [ ]:
import random
from scipy.stats import norm
import numpy as np
import math

# Galton board function
def simulate_galton_board(asset_price, drift, volatility, time_to_expiry, num_simulations, num_slots):
    if not isinstance(asset_price, (float, int)):
        raise TypeError('asset_price must be a numeric value')

    dt = time_to_expiry / num_slots
    asset_prices = [asset_price] * num_simulations

    for i in range(num_simulations):
        for j in range(num_slots):
            asset_prices[i] *= (1 + (drift - 0.5 * volatility ** 2) * dt + volatility * norm.ppf(random.random()) * (dt ** 0.5))

    return asset_prices


def payoff(asset_price, strike_price, option_type):
    if option_type == 'call':
        return max(asset_price - strike_price, 0)
    elif option_type == 'put':
        return max(strike_price - asset_price, 0)
    else:
        raise ValueError('option_type must be either "call" or "put"')


def option_price(asset_price, strike_price, risk_free_rate, time_to_expiry, option_type, num_simulations=10000, num_slots=100):
    dt = time_to_expiry / num_slots
    asset_prices = [asset_price] * num_simulations
    for i in range(num_slots):
        slots = simulate_galton_board(asset_price, drift, volatility, time_to_expiry, num_simulations, num_slots)
        for j in range(num_simulations):
            asset_prices[j] *= (1 + (drift - 0.5 * volatility ** 2) * dt + volatility * norm.ppf(slots[j] / num_simulations) * (dt ** 0.5))
    payoffs = [payoff(ap, strike_price, option_type) for ap in asset_prices]
    discount_factor = math.exp(-risk_free_rate * time_to_expiry)
    return discount_factor * sum(payoffs) / num_simulations


# Example usage:
asset_price = 39.8
strike_price = 39.5
risk_free_rate = 0.02
time_to_expiry = 0.5
num_simulations = 10000
num_slots = 100
drift = 0.1
volatility = 0.2
option_type = "put"

# Calculate the option price based on the estimated probability distribution
estimated_price = option_price(asset_price, strike_price, risk_free_rate, time_to_expiry, option_type, num_simulations, num_slots)

# Print the result
print("Estimated Price: ", estimated_price)
